In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.dynamical_systems import dare, moas_closed_loop
from pympc.control import MPCHybridController
from pympc.feasible_set_library import FeasibleSetLibrary, load_library
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt
import director.viewerclient as vc
from director.thirdparty import transformations

# Construction of Box-Atlas

Robot moving limbs

In [ ]:
from pympc.models.boxatlas_parameters import weight, stiffness
penetration = weight / stiffness

# left hand
A_lh = {
    'not_in_contact': np.array([[1., 0.]]),
    'in_contact': np.array([[-1., 0.],[1., 0.]]),
}
b_lh = {
    'not_in_contact': np.array([[.5]]),
    'in_contact': np.array([[-.5],[.5+penetration]]),
}
contact_surfaces_lh = {
    'not_in_contact': None,
    'in_contact': 0,
}
q_lh = np.array([[.3], [.0]])
lh = MovingLimb(A_lh, b_lh, contact_surfaces_lh, q_lh)

# right hand
A_rh = {
    'not_in_contact_side': np.array([[-1., 0.],[-1., 1.]]),
    'not_in_contact_top': np.array([[0., -1.],[1., -1.]]),
    'in_contact_side': np.array([[1., 0.],[-1., 0.],[-1., 1.]]),
    'in_contact_top': np.array([[0., 1.],[0., -1.],[1., -1.]]),
}
b_rh = {
    'not_in_contact_side': np.array([[.4],[.4]]),
    'not_in_contact_top': np.array([[0.],[-.4]]),
    'in_contact_side': np.array([[-.4],[.4 + penetration],[.4]]),
    'in_contact_top': np.array([[0.],[penetration],[-.4]]),
}
contact_surfaces_rh = {
    'not_in_contact_side': None,
    'not_in_contact_top': None,
    'in_contact_side': 0,
    'in_contact_top': 0,
}
q_rh = np.array([[-.3], [.0]])
forbidden_transitions_rh = [
    ('not_in_contact_side', 'in_contact_top'),
    ('in_contact_top', 'not_in_contact_side'),
    ('not_in_contact_top', 'in_contact_side'),
    ('in_contact_side', 'not_in_contact_top'),
    ('in_contact_side', 'in_contact_top'),
    ('in_contact_top', 'in_contact_side')
]
rh = MovingLimb(A_rh, b_rh, contact_surfaces_rh, q_rh, forbidden_transitions_rh)

Robot fixed limbs

In [ ]:
from pympc.models.boxatlas_parameters import weight

# left foot
q_lf = np.array([[.2], [-.5]])
n_lf = np.array([[0.],[1.]])
f_lf = np.array([[weight / 2.], [0.]])
lf = FixedLimb(q_lf, n_lf, f_lf)

# right foot
q_rf = np.array([[-.2], [-.5]])
n_rf = np.array([[0.],[1.]])
f_rf = np.array([[weight / 2.], [0.]])
rf = FixedLimb(q_rf, n_rf, f_rf)

Assemble robot

In [ ]:
limbs = { 'moving': {'lh': lh, 'rh': rh}, 'fixed': {'lf': lf, 'rf': rf} }
nominal_mode = {'rh': 'not_in_contact_side', 'lh': 'not_in_contact'}
box_atlas = BoxAtlas(limbs, nominal_mode)

Print state, input, and modes

In [ ]:
box_atlas.print_state_labels()
box_atlas.print_input_labels()
print 'Box-atlas modes:\n', box_atlas.contact_modes

# Hybrid MPC Controller

Hybrid MPC controller

In [ ]:
N = 10
objective_norm = 'two'
controller = MPCHybridController(
    box_atlas.pwa_system,
    N,
    objective_norm,
    box_atlas.Q,
    box_atlas.R,
    box_atlas.P,
    box_atlas.X_N)

Trick to avoid forbidden transitions

In [ ]:
controller = box_atlas.avoid_forbidden_transitions(controller)

Closed-loop system simulation of the MPC controller

In [ ]:
N_sim = 30
u = []
x_0 = np.array([
    [.0], # q_rh_x
    [.0], # q_rh_y
    [.0], # q_lh_x
    [.0], # q_lh_y
    [.0], # q_b_x
    [.0], # q_b_y
    [-1.], # v_b_x
    [-.3] # v_b_y
])
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
for k in range(N_sim):
    u_k, x_k, ss_k = controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print('Time step ' + str(k) + ': ' + str(ss_k) + '.')
    u_ws = u_k[1:] + [K.dot(x_k[-1])]
    x_ws = x_k[1:] + [box_atlas.pwa_system.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (terminal_mode,)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k[0]])[0][1]
    #print 'Planner-simulator missmatch:', np.linalg.norm(x_next - x_k[1])
    u.append(u_k[0])
    x.append(x_next)
traj_miqp = Trajectory(x, u, Q, R, P)
#np.save('box_atlas_tajectory_miqp_lh', traj_miqp)

Plot of the state and the input

In [ ]:
mpc_plt.input_sequence(u, parameters['sampling_time'])
plt.show()
mpc_plt.state_trajectory(x, parameters['sampling_time'])
plt.show()

# Control with Approximate Hybrid MPC

Initialization of the library

In [ ]:
library = FeasibleSetLibrary(controller)
samples_so_far = 0
#library = load_library('library_box_atlas_wall_and_table_' + str(samples_so_far))

Coverage

In [ ]:
sample_step = 2
n_steps = 1
for i in range(n_steps):
    n = samples_so_far + (i+1)*sample_step
    library.sample_policy(sample_step, box_atlas.is_inside_a_domain)
    print(str(n) + ' samples added to the library.')
    #library.save('library_box_atlas_wall_and_table_' + str(n))

Shifted switching sequences

In [ ]:
library.add_shifted_switching_sequences(terminal_mode)

Closed-loop system simulation of the approximate MPC controller

In [ ]:
N_sim = 50
u = []
x = [x_0_lh*0.]
ss = []
ss_feasible = None
for k in range(N_sim):
    print('Time step ' + str(k) + '.')
    u_k, ss_k = library.feedback(x[k], ss_feasible, max_qp=1)
    u.append(u_k)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k])[0][1]
    x.append(x_next)
    ss.append(ss_k)
    ss_feasible = ss_k[1:] + (terminal_mode,)
traj_lib = Trajectory(x, u, Q, R, P)
#np.save('box_atlas_tajectory_lib_lh', traj)

# Test Coverage

Number of feasible sets in the library

In [ ]:
n_fs = 0
for ss in library.library.values():
    if not ss['feasible_set'].empty:
        n_fs += 1
print 'Number of mode sequences in the library:', n_fs

Check coverage of the feasible set with n_samples

In [ ]:
n_samples = 20
cost_miqp = []
cost_qp = []
for i in range(n_samples):
    print i
    x_sample = library.random_sample(box_atlas.is_inside_a_domain)
    cost_miqp.append(controller.feedforward(x_sample)[3])
    cost_qp.append(library.feedforward(x_sample)[3])
#np.save('cost_random_miqp', cost_miqp)
#np.save('cost_random_qp', cost_qp)

Cost error as a function of the maximum number of QPs online

In [ ]:
feasible_samples = 0
samples_not_covered = 0
optimality_loss = []
for i, c_mi in enumerate(cost_miqp):
    if not np.isnan(c_mi):
        feasible_samples += 1
        if len(cost_qp[i]) == 0:
            samples_not_covered += 1
        else:
            optimality_loss.append([(c - c_mi)/c_mi for c in cost_qp[i]])
max_overlapping_qp = max([len(ol) for ol in optimality_loss])
loss = []
for pos in range(max_overlapping_qp):
    average_loss = 0
    for ol in optimality_loss:
        length = min(len(ol),pos+1)
        average_loss += min(ol[:length])
    loss.append(average_loss/float(len(optimality_loss))*100)
print 'Optimality loss (%) as a function of the maximum number of QPs solved online:\n', loss

## Visualizer

In [ ]:
box_atlas.visualize(x[0])
time.sleep(10 * parameters['sampling_time'])
for k in range(N_sim):
    box_atlas.visualize(x[k])
    time.sleep(parameters['sampling_time'])

In [ ]:
# test
x_0_lh = np.array([
    [0.], # q_rh_x
    [0.], # q_rh_y
    [0.], # q_lh_x
    [0.], # q_lh_y
    [0.], # q_b_x
    [0.], # q_b_y
    [0.], # t_b
    [-1.], # v_b_x
    [-.5], # v_b_y
    [0.] # o_b
])
u, x, ss = controller.feedforward(x_0_lh)[0:3]

In [ ]:
from pympc.models.boxatlas_parameters import sampling_time

box_atlas.visualize(x[0])
time.sleep(20 * sampling_time)
for k in range(N):
    box_atlas.visualize(x[k])
    time.sleep(sampling_time)

In [ ]:
print ss
box_atlas.print_mode_sequence(ss)